# ESKAR Housing Finder - ML Data Analysis
## Code Institute PP5 - Comprehensive Data Science Notebook

**Project:** ESKAR Housing Finder - AI-powered housing recommendation system for ESK personnel

**Author:** Student Name  
**Date:** August 2025  
**Institution:** Code Institute

---

### Business Requirements
- **BR1:** Data Visualization and Correlation Analysis
- **BR2:** ML Pipeline for ESK Suitability Prediction
- **BR3:** Interactive Dashboard for End Users

### Project Objectives
1. Analyze housing market data in Karlsruhe area
2. Develop ML models to predict ESK suitability scores
3. Create deployment-ready model pipeline
4. Validate business hypotheses with data

## 1. Data Collection and Setup

In [ ]:
# Import Essential Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ML Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb

# Configuration
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("[SUCCESS] Libraries imported successfully")
print(f"[VERSION] Pandas version: {pd.__version__}")
print(f"[VERSION] XGBoost version: {xgb.__version__}")

📊 Libraries imported successfully
📈 Pandas version: 2.3.1
🤖 XGBoost version: 3.0.4


## 2. Data Loading and Initial Exploration

In [ ]:
# Load housing data
import sys
sys.path.append('../')
from data_generator import ESKARDataGenerator

# Generate comprehensive dataset
generator = ESKARDataGenerator()
df = generator.generate_housing_dataset(n_samples=1000)

print(f"[DATA] Dataset Shape: {df.shape}")
print(f"[FEATURES] Features: {list(df.columns)}")
print("\n[INFO] Dataset Info:")
df.info()

print("\n[SAMPLE] First 5 rows:")
df.head()

🏫 Generating 1000 ESK-optimized properties...
✅ 1000 ESK-optimized properties generated!
📊 Average ESK Score: 8.0/10
🏠 412 houses, 588 apartments
🎯 400 properties within 2km of ESK

📈 Neighborhood Distribution:
   Südstadt: 254 properties (⌀ €453,528)
   Weststadt: 380 properties (⌀ €494,451)
   Innenstadt-West: 141 properties (⌀ €576,204)
   Durlach: 114 properties (⌀ €409,852)
   Oststadt: 66 properties (⌀ €440,357)
   Mühlburg: 45 properties (⌀ €344,328)
📋 Dataset Shape: (1000, 20)
📊 Features: ['id', 'neighborhood', 'property_type', 'bedrooms', 'sqft', 'garden', 'price', 'price_per_sqm', 'distance_to_esk', 'distance_to_center', 'avg_employer_distance', 'esk_suitability_score', 'safety_score', 'international_community_score', 'family_amenities_score', 'public_transport_score', 'current_esk_families', 'commute_time_esk', 'lat', 'lon']

📝 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                  

,id,neighborhood,property_type,bedrooms,sqft,garden,price,price_per_sqm,distance_to_esk,distance_to_center,avg_employer_distance,esk_suitability_score,safety_score,international_community_score,family_amenities_score,public_transport_score,current_esk_families,commute_time_esk,lat,lon
0,ESKAR_001,Südstadt,apartment,4,62,0,291327,4683,1.5,1.1,10.4,8.3,8.8,8.2,8.5,9.0,38,15,49.003189,8.417388
1,ESKAR_002,Weststadt,apartment,4,75,0,237331,3144,2.4,2.0,11.0,8.4,9.2,8.5,9.0,8.8,45,12,49.001648,8.377329
2,ESKAR_003,Südstadt,house,4,123,1,498610,4058,2.5,1.8,11.3,8.3,8.8,8.2,8.5,9.0,38,15,48.992576,8.415685
3,ESKAR_004,Weststadt,apartment,3,93,0,392419,4217,3.2,2.9,11.6,7.8,9.2,8.5,9.0,8.8,45,12,49.001746,8.364690
4,ESKAR_005,Innenstadt-West,apartment,2,77,1,315607,4117,0.7,0.2,9.9,8.3,8.5,7.5,7.8,9.5,28,8,49.006845,8.406038


## 3. Data Quality Assessment

In [4]:
# Data Quality Check
print("[ASSESSMENT] Data Quality Assessment\n")

# Missing values
missing_data = df.isnull().sum()
print("[MISSING] Missing Values:")
print(missing_data[missing_data > 0])

# Data types
print("\n[TYPES] Data Types:")
print(df.dtypes)

# Statistical summary
print("\n[SUMMARY] Statistical Summary:")
df.describe()

[ASSESSMENT] Data Quality Assessment

[MISSING] Missing Values:
Series([], dtype: int64)

[TYPES] Data Types:
id                                object
neighborhood                      object
property_type                     object
bedrooms                           int64
sqft                               int64
garden                             int64
price                              int64
price_per_sqm                      int64
distance_to_esk                  float64
distance_to_center               float64
avg_employer_distance            float64
esk_suitability_score            float64
safety_score                     float64
international_community_score    float64
family_amenities_score           float64
public_transport_score           float64
current_esk_families               int64
commute_time_esk                   int64
lat                              float64
lon                              float64
dtype: object

[SUMMARY] Statistical Summary:


,bedrooms,sqft,garden,price,price_per_sqm,distance_to_esk,distance_to_center,avg_employer_distance,esk_suitability_score,safety_score,international_community_score,family_amenities_score,public_transport_score,current_esk_families,commute_time_esk,lat,lon
count,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,3.254000,110.038000,0.523000,4.756140e+05,4318.352000,2.651400,2.454500,11.018100,8.017300,8.863500,7.949500,8.624600,8.703500,34.738000,14.436000,49.004581,8.395553
std,0.887847,39.168773,0.499721,1.919239e+05,788.910818,1.477519,1.495445,1.052602,0.464442,0.345597,0.599382,0.474552,0.591767,10.803429,4.811762,0.010694,0.034758
min,2.000000,50.000000,0.000000,1.170230e+05,2120.000000,0.100000,0.100000,9.100000,6.700000,8.000000,6.800000,7.800000,7.500000,12.000000,8.000000,48.964611,8.318407
25%,3.000000,81.000000,0.000000,3.313252e+05,3774.250000,1.600000,1.400000,10.200000,7.800000,8.500000,7.500000,8.200000,8.800000,28.000000,12.000000,48.997705,8.373120
50%,3.000000,104.000000,1.000000,4.433950e+05,4311.500000,2.300000,2.100000,10.900000,8.200000,8.800000,8.200000,8.500000,8.800000,38.000000,12.000000,49.004141,8.388413
75%,4.000000,133.000000,1.000000,5.822060e+05,4827.750000,3.400000,3.200000,11.700000,8.400000,9.200000,8.500000,9.000000,9.000000,45.000000,15.000000,49.011977,8.406044
max,5.000000,256.000000,1.000000,1.312292e+06,6677.000000,8.400000,8.300000,14.600000,8.800000,9.200000,8.500000,9.200000,9.500000,45.000000,25.000000,49.034560,8.513268


## 4. Exploratory Data Analysis (EDA)

In [7]:
# Price Distribution Analysis
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Price Distribution', 'Price by Property Type', 
                   'Price by Neighborhood', 'Distance to ESK vs Price'),
    specs=[[{"type": "histogram"}, {"type": "box"}],
           [{"type": "box"}, {"type": "scatter"}]]
)

# Price histogram
fig.add_trace(
    go.Histogram(x=df['price'], name='Price', nbinsx=30),
    row=1, col=1
)

# Price by property type
for prop_type in df['property_type'].unique():
    fig.add_trace(
        go.Box(y=df[df['property_type']==prop_type]['price'], 
               name=prop_type),
        row=1, col=2
    )

# Price by neighborhood (top 10)
top_neighborhoods = df['neighborhood'].value_counts().head(10).index
df_top = df[df['neighborhood'].isin(top_neighborhoods)]
for neighborhood in top_neighborhoods:
    fig.add_trace(
        go.Box(y=df_top[df_top['neighborhood']==neighborhood]['price'], 
               name=neighborhood),
        row=2, col=1
    )

# Distance vs Price scatter
fig.add_trace(
    go.Scatter(x=df['distance_to_esk'], y=df['price'], 
               mode='markers', name='Properties',
               marker=dict(color=df['esk_suitability_score'], 
                          colorscale='Viridis', showscale=True)),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Housing Market Analysis")

# Display figure (Notebook compatible)
from IPython.display import display
display(fig)

print("[INSIGHTS] Key Insights:")
print(f"[PRICE] Average Price: €{df['price'].mean():,.2f}")
print(f"[DISTANCE] Average Distance to ESK: {df['distance_to_esk'].mean():.2f} km")
print(f"[SCORE] Average ESK Suitability: {df['esk_suitability_score'].mean():.1f}/100")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'name': 'Price',
              'nbinsx': 30,
              'type': 'histogram',
              'x': {'bdata': ('/3EEABOfAwCymwcA4/wFANfQBAAMbQ' ... 'CUpQoA3kwDAALaAwA16AMAFrQFAA=='),
                    'dtype': 'i4'},
              'xaxis': 'x',
              'yaxis': 'y'},
             {'name': np.str_('apartment'),
              'type': 'box',
              'xaxis': 'x2',
              'y': {'bdata': ('/3EEABOfAwDj/AUA19AEAAxtBgCNtQ' ... 'MFABj1AgCUpQoA3kwDAALaAwAWtAUA'),
                    'dtype': 'i4'},
              'yaxis': 'y2'},
             {'name': np.str_('house'),
              'type': 'box',
              'xaxis': 'x2',
              'y': {'bdata': ('spsHAH4zBQAFsAoAwTMGAAv+BwAPzg' ... 'CEgwgAIwAIAFqyCwCREw8ANegDAA=='),
                    'dtype': 'i4'},
              'yaxis': 'y2'},
             {'name': np.str_('Weststadt'),
              'type': 'box',
              'xaxis': 'x3',
              'y': {'bdata': ('E58DAOP8BQAMbQYAjbUGAMEZAwAL/g' ... 'UAIwAIAKwDBQCUpQoA3kwDAALaAwA='),
                    'dtype': 'i4'},
              'yaxis': 'y3'},
             {'name': np.str_('Südstadt'),
              'type': 'box',
              'xaxis': 'x3',
              'y': {'bdata': ('/3EEALKbBwB+MwUAlK4DABGyBQCe/A' ... 'cAN80IAISDCABasgsANegDABa0BQA='),
                    'dtype': 'i4'},
              'yaxis': 'y3'},
             {'name': np.str_('Innenstadt-West'),
              'type': 'box',
              'xaxis': 'x3',
              'y': {'bdata': ('19AEAA/OBwCtXgoAZR8IANIbCgBeJg' ... 'gFALwKCgBwMAgA78EGAP5+CACREw8A'),
                    'dtype': 'i4'},
              'yaxis': 'y3'},
             {'name': np.str_('Durlach'),
              'type': 'box',
              'xaxis': 'x3',
              'y': {'bdata': ('56cFAAWwCgCYBgUAy3YJAGdUBQB89w' ... 'AGAC1nAwD37woAJvsGAFS1AgAY9QIA'),
                    'dtype': 'i4'},
              'yaxis': 'y3'},
             {'name': np.str_('Oststadt'),
              'type': 'box',
              'xaxis': 'x3',
              'y': {'bdata': ('C5gIAMEzBgCeFgQAxiQGALZfBQDFvg' ... 'EDAH6CBACPVwgAWs8DANDsCABpFAgA'),
                    'dtype': 'i4'},
              'yaxis': 'y3'},
             {'name': np.str_('Mühlburg'),
              'type': 'box',
              'xaxis': 'x3',
              'y': {'bdata': ('WnMJAJLGBgD9bQUAJT0GANrJBgDdEQ' ... 'cCAB6bBQCbKwQAuv4GABL+BACKdgMA'),
                    'dtype': 'i4'},
              'yaxis': 'y3'},
             {'marker': {'color': {'bdata': ('mpmZmZmZIEDNzMzMzMwgQJqZmZmZmS' ... 'MzMzMfQJqZmZmZmSBAZmZmZmZmIEA='),
                                   'dtype': 'f8'},
                         'colorscale': [[0.0, '#440154'], [0.1111111111111111,
                                        '#482878'], [0.2222222222222222,
                                        '#3e4989'], [0.3333333333333333,
                                        '#31688e'], [0.4444444444444444,
                                        '#26828e'], [0.5555555555555556,
                                        '#1f9e89'], [0.6666666666666666,
                                        '#35b779'], [0.7777777777777778,
                                        '#6ece58'], [0.8888888888888888,
                                        '#b5de2b'], [1.0, '#fde725']],
                         'showscale': True},
              'mode': 'markers',
              'name': 'Properties',
              'type': 'scatter',
              'x': {'bdata': ('AAAAAAAA+D8zMzMzMzMDQAAAAAAAAA' ... 'mZmZkRQAAAAAAAAARAZmZmZmZm9j8='),
                    'dtype': 'f8'},
              'xaxis': 'x4',
              'y': {'bdata': ('/3EEABOfAwCymwcA4/wFANfQBAAMbQ' ... 'CUpQoA3kwDAALaAwA16AMAFrQFAA=='),
                    'dtype': 'i4'},
              'yaxis': 'y4'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Price Distribution',
                                'x': 0.225,
                     

[INSIGHTS] Key Insights:
[PRICE] Average Price: €475,613.99
[DISTANCE] Average Distance to ESK: 2.65 km
[SCORE] Average ESK Suitability: 8.0/100


## 5. Feature Correlation Analysis

In [9]:
# Correlation Matrix for Available Numerical Features
print("[FEATURES] Available columns:")
print(df.columns.tolist())

# Use available numerical features
numerical_features = ['price', 'distance_to_esk', 'esk_suitability_score', 'bedrooms', 'sqft']

# Use available binary features (convert to numeric)
df['garden_numeric'] = df['garden'].astype(int)
binary_features = ['garden_numeric']

# Check what other features are available
if 'safety_score' in df.columns:
    numerical_features.append('safety_score')
if 'family_amenities_score' in df.columns:
    numerical_features.append('family_amenities_score')

all_features = numerical_features + binary_features

print(f"[CORRELATION] Using features: {all_features}")

# Calculate correlation matrix
corr_matrix = df[all_features].corr()

# Create interactive heatmap
fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='RdBu',
    zmid=0,
    text=corr_matrix.round(3).values,
    texttemplate="%{text}",
    textfont={"size":10}
))

fig.update_layout(
    title="Feature Correlation Matrix",
    width=800,
    height=600
)

# Display with fallback
try:
    from IPython.display import display
    display(fig)
except:
    print("[PLOT] Correlation matrix calculated successfully")
    print(corr_matrix)

# Key correlations with ESK suitability
esk_correlations = corr_matrix['esk_suitability_score'].sort_values(ascending=False)
print("[CORRELATION] Features most correlated with ESK Suitability:")
print(esk_correlations)

[FEATURES] Available columns:
['id', 'neighborhood', 'property_type', 'bedrooms', 'sqft', 'garden', 'price', 'price_per_sqm', 'distance_to_esk', 'distance_to_center', 'avg_employer_distance', 'esk_suitability_score', 'safety_score', 'international_community_score', 'family_amenities_score', 'public_transport_score', 'current_esk_families', 'commute_time_esk', 'lat', 'lon']
[CORRELATION] Using features: ['price', 'distance_to_esk', 'esk_suitability_score', 'bedrooms', 'sqft', 'safety_score', 'family_amenities_score', 'garden_numeric']


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'colorscale': [[0.0, 'rgb(103,0,31)'], [0.1, 'rgb(178,24,43)'],
                             [0.2, 'rgb(214,96,77)'], [0.3, 'rgb(244,165,130)'],
                             [0.4, 'rgb(253,219,199)'], [0.5, 'rgb(247,247,247)'],
                             [0.6, 'rgb(209,229,240)'], [0.7, 'rgb(146,197,222)'],
                             [0.8, 'rgb(67,147,195)'], [0.9, 'rgb(33,102,172)'],
                             [1.0, 'rgb(5,48,97)']],
              'text': {'bdata': ('AAAAAAAA8D9I4XoUrkfBvzMzMzMzM8' ... '5H4XqUP/yp8dJNYlA/AAAAAAAA8D8='),
                       'dtype': 'f8',
                       'shape': '8, 8'},
              'textfont': {'size': 10},
              'texttemplate': '%{text}',
              'type': 'heatmap',
              'x': array(['price', 'distance_to_esk', 'esk_suitability_score', 'bedrooms', 'sqft',
                          'safety_score', 'family_amenities_score', 'garden_numeric'],
                         dtype=object),
              'y': array(['price', 'distance_to_esk', 'esk_suitability_score', 'bedrooms', 'sqft',
                          'safety_score', 'family_amenities_score', 'garden_numeric'],
                         dtype=object),
              'z': {'bdata': ('AAAAAAAA8D+BPG018UnBv2IjHrjdK8' ... '6fAraUPx+oIVnVHFc/AAAAAAAA8D8='),
                    'dtype': 'f8',
                    'shape': '8, 8'},
              'zmid': 0}],
    'layout': {'height': 600, 'template': '...', 'title': {'text': 'Feature Correlation Matrix'}, 'width': 800}
})

[CORRELATION] Features most correlated with ESK Suitability:
esk_suitability_score     1.000000
price                     0.149776
safety_score              0.127793
garden_numeric            0.093816
bedrooms                  0.069441
sqft                     -0.011008
family_amenities_score   -0.187552
distance_to_esk          -0.861078
Name: esk_suitability_score, dtype: float64


## 6. ML Pipeline Development

In [11]:
# Feature Engineering
def prepare_features(df):
    """Prepare features for ML modeling"""
    df_processed = df.copy()
    
    # Create new features using available columns
    if 'price_per_sqm' not in df_processed.columns:
        df_processed['price_per_sqm'] = df_processed['price'] / df_processed['sqft']
    
    # Use available columns for total rooms (no bathrooms column available)
    df_processed['total_rooms'] = df_processed['bedrooms']  # Only bedrooms available
    
    # Feature score using available amenities
    df_processed['feature_score'] = df_processed['garden'].astype(int)
    
    # Add additional available features
    if 'safety_score' in df_processed.columns:
        df_processed['safety_normalized'] = df_processed['safety_score'] / 10.0
    if 'family_amenities_score' in df_processed.columns:
        df_processed['family_amenities_normalized'] = df_processed['family_amenities_score'] / 10.0
    
    # Encode categorical variables
    le_neighborhood = LabelEncoder()
    le_property_type = LabelEncoder()
    
    df_processed['neighborhood_encoded'] = le_neighborhood.fit_transform(df_processed['neighborhood'])
    df_processed['property_type_encoded'] = le_property_type.fit_transform(df_processed['property_type'])
    
    return df_processed, le_neighborhood, le_property_type

# Prepare data
df_processed, le_neighborhood, le_property_type = prepare_features(df)

print("[SUCCESS] Feature Engineering Complete")
print(f"[NEW] New Features: {[col for col in df_processed.columns if col not in df.columns]}")
print(f"[TOTAL] Total Features: {df_processed.shape[1]}")
print(f"[AVAILABLE] Key features for ML: {[col for col in df_processed.columns if col in ['price', 'distance_to_esk', 'bedrooms', 'sqft', 'garden', 'safety_score']]}")

[SUCCESS] Feature Engineering Complete
[NEW] New Features: ['total_rooms', 'feature_score', 'safety_normalized', 'family_amenities_normalized', 'neighborhood_encoded', 'property_type_encoded']
[TOTAL] Total Features: 27
[AVAILABLE] Key features for ML: ['bedrooms', 'sqft', 'garden', 'price', 'distance_to_esk', 'safety_score']


## 7. Model Training and Evaluation

In [12]:
# Define features and target using available columns
feature_columns = ['distance_to_esk', 'bedrooms', 'sqft', 'price_per_sqm', 
                  'total_rooms', 'feature_score', 'safety_normalized',
                  'family_amenities_normalized', 'neighborhood_encoded', 'property_type_encoded']

X = df_processed[feature_columns]
y = df_processed['esk_suitability_score']

print(f"[FEATURES] Using features: {feature_columns}")
print(f"[TARGET] Target variable: esk_suitability_score")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"[TRAINING] Training set: {X_train.shape}")
print(f"[TEST] Test set: {X_test.shape}")

# Initialize models
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
}

# Train and evaluate models
results = {}

for name, model in models.items():
    print(f"\n[TRAINING] Training {name}...")
    
    # Train model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'model': model,
        'predictions': y_pred
    }
    
    print(f"[RESULTS] {name} Results:")
    print(f"   MAE: {mae:.3f}")
    print(f"   RMSE: {rmse:.3f}")
    print(f"   R²: {r2:.3f}")

# Find best model
best_model_name = min(results.keys(), key=lambda k: results[k]['RMSE'])
best_model = results[best_model_name]['model']

print(f"\n[WINNER] Best Model: {best_model_name}")
print(f"[SCORE] Best RMSE: {results[best_model_name]['RMSE']:.3f}")
print(f"[ACCURACY] Best R²: {results[best_model_name]['R²']:.3f}")

[FEATURES] Using features: ['distance_to_esk', 'bedrooms', 'sqft', 'price_per_sqm', 'total_rooms', 'feature_score', 'safety_normalized', 'family_amenities_normalized', 'neighborhood_encoded', 'property_type_encoded']
[TARGET] Target variable: esk_suitability_score
[TRAINING] Training set: (800, 10)
[TEST] Test set: (200, 10)

[TRAINING] Training Random Forest...
[RESULTS] Random Forest Results:
   MAE: 0.045
   RMSE: 0.074
   R²: 0.975

[TRAINING] Training XGBoost...
[RESULTS] XGBoost Results:
   MAE: 0.049
   RMSE: 0.079
   R²: 0.972

[TRAINING] Training LightGBM...
[RESULTS] XGBoost Results:
   MAE: 0.049
   RMSE: 0.079
   R²: 0.972

[TRAINING] Training LightGBM...
[RESULTS] LightGBM Results:
   MAE: 0.048
   RMSE: 0.072
   R²: 0.976

[WINNER] Best Model: LightGBM
[SCORE] Best RMSE: 0.072
[ACCURACY] Best R²: 0.976
[RESULTS] LightGBM Results:
   MAE: 0.048
   RMSE: 0.072
   R²: 0.976

[WINNER] Best Model: LightGBM
[SCORE] Best RMSE: 0.072
[ACCURACY] Best R²: 0.976


## 8. Model Performance Visualization

In [ ]:
# Model Comparison
model_names = list(results.keys())
mae_scores = [results[name]['MAE'] for name in model_names]
rmse_scores = [results[name]['RMSE'] for name in model_names]
r2_scores = [results[name]['R²'] for name in model_names]

# Create comparison plot
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Mean Absolute Error', 'Root Mean Square Error', 'R² Score'),
    specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "bar"}]]
)

# MAE
fig.add_trace(
    go.Bar(x=model_names, y=mae_scores, name='MAE', marker_color='lightblue'),
    row=1, col=1
)

# RMSE
fig.add_trace(
    go.Bar(x=model_names, y=rmse_scores, name='RMSE', marker_color='lightcoral'),
    row=1, col=2
)

# R²
fig.add_trace(
    go.Bar(x=model_names, y=r2_scores, name='R²', marker_color='lightgreen'),
    row=1, col=3
)

fig.update_layout(height=400, title_text="Model Performance Comparison", showlegend=False)
fig.show()

# Prediction vs Actual for best model
best_predictions = results[best_model_name]['predictions']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=y_test, 
    y=best_predictions,
    mode='markers',
    name='Predictions',
    marker=dict(color='blue', opacity=0.6)
))

# Perfect prediction line
min_val = min(y_test.min(), best_predictions.min())
max_val = max(y_test.max(), best_predictions.max())
fig.add_trace(go.Scatter(
    x=[min_val, max_val], 
    y=[min_val, max_val],
    mode='lines',
    name='Perfect Prediction',
    line=dict(color='red', dash='dash')
))

fig.update_layout(
    title=f"{best_model_name} - Predicted vs Actual ESK Suitability Scores",
    xaxis_title="Actual ESK Suitability Score",
    yaxis_title="Predicted ESK Suitability Score",
    width=600,
    height=500
)
fig.show()

## 9. Feature Importance Analysis

In [15]:
# Feature importance for best model
feature_importance = best_model.feature_importances_
feature_names = feature_columns

# Create feature importance dataframe
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=True)

print("[FEATURE IMPORTANCE] Feature Importance Analysis")
print("=" * 50)

# Display feature importance table
print("\n[TOP FEATURES] Top 5 Most Important Features:")
print(importance_df.tail().to_string(index=False))

print("\n[ALL FEATURES] Complete Feature Ranking:")
for idx, row in importance_df.iterrows():
    print(f"   {row['feature']:25s}: {row['importance']:.4f}")

# Business interpretation
print("\n[INTERPRETATION] Business Interpretation:")
top_feature = importance_df.iloc[-1]['feature']
second_feature = importance_df.iloc[-2]['feature']
print(f"   • Most important factor: {top_feature}")
print(f"   • Second most important: {second_feature}")
print("   • Distance to ESK is likely the key driver of suitability")
print("   • Property characteristics (size, type) also matter significantly")

[FEATURE IMPORTANCE] Feature Importance Analysis

[TOP FEATURES] Top 5 Most Important Features:
          feature  importance
safety_normalized         238
         bedrooms         273
             sqft         615
    price_per_sqm         743
  distance_to_esk         749

[ALL FEATURES] Complete Feature Ranking:
   total_rooms              : 0.0000
   family_amenities_normalized: 8.0000
   property_type_encoded    : 44.0000
   neighborhood_encoded     : 46.0000
   feature_score            : 203.0000
   safety_normalized        : 238.0000
   bedrooms                 : 273.0000
   sqft                     : 615.0000
   price_per_sqm            : 743.0000
   distance_to_esk          : 749.0000

[INTERPRETATION] Business Interpretation:
   • Most important factor: distance_to_esk
   • Second most important: price_per_sqm
   • Distance to ESK is likely the key driver of suitability
   • Property characteristics (size, type) also matter significantly


## 10. Business Insights and Conclusions

In [16]:
# Business Insights Analysis
print("[SUMMARY] BUSINESS INSIGHTS SUMMARY")
print("=" * 50)

print("\n[PERFORMANCE] MODEL PERFORMANCE:")
print(f"   • Best Model: {best_model_name}")
print(f"   • Prediction Accuracy (R²): {results[best_model_name]['R²']:.1%}")
print(f"   • Average Prediction Error: ±{results[best_model_name]['MAE']:.1f} points")

print("\n[MARKET] KEY HOUSING MARKET FINDINGS:")
print(f"   • Average Property Price: €{df['price'].mean():,.0f}")
print(f"   • Price Range: €{df['price'].min():,.0f} - €{df['price'].max():,.0f}")
print(f"   • Most Common Property Type: {df['property_type'].mode()[0]}")
print(f"   • Average Distance to ESK: {df['distance_to_esk'].mean():.1f} km")

print("\n[FACTORS] ESK SUITABILITY FACTORS:")
top_features = importance_df.tail(3)['feature'].tolist()
for i, feature in enumerate(top_features, 1):
    print(f"   {i}. {feature.replace('_', ' ').title()}")

print("\n[RECOMMENDATIONS] BUSINESS RECOMMENDATIONS:")
print("   • Focus on properties within 5km of ESK for higher suitability")
print("   • Prioritize properties with gardens and parking for families")
print("   • Consider price-per-sqm ratio for value optimization")
print("   • Target specific neighborhoods with high ESK scores")

print("\n[DEPLOYMENT] DEPLOYMENT READINESS:")
print("   • Model successfully trained and validated")
print("   • Feature pipeline established")
print("   • Ready for Streamlit integration")
print("   • API-ready prediction function available")

[SUMMARY] BUSINESS INSIGHTS SUMMARY

[PERFORMANCE] MODEL PERFORMANCE:
   • Best Model: LightGBM
   • Prediction Accuracy (R²): 97.6%
   • Average Prediction Error: ±0.0 points

[MARKET] KEY HOUSING MARKET FINDINGS:
   • Average Property Price: €475,614
   • Price Range: €117,023 - €1,312,292
   • Most Common Property Type: apartment
   • Average Distance to ESK: 2.7 km

[FACTORS] ESK SUITABILITY FACTORS:
   1. Sqft
   2. Price Per Sqm
   3. Distance To Esk

[RECOMMENDATIONS] BUSINESS RECOMMENDATIONS:
   • Focus on properties within 5km of ESK for higher suitability
   • Prioritize properties with gardens and parking for families
   • Consider price-per-sqm ratio for value optimization
   • Target specific neighborhoods with high ESK scores

[DEPLOYMENT] DEPLOYMENT READINESS:
   • Model successfully trained and validated
   • Feature pipeline established
   • Ready for Streamlit integration
   • API-ready prediction function available


## 11. Model Export for Production

In [17]:
# Save the best model for production use
import joblib
import os

# Create models directory
os.makedirs('../models', exist_ok=True)

# Save model and encoders
model_artifacts = {
    'model': best_model,
    'feature_columns': feature_columns,
    'neighborhood_encoder': le_neighborhood,
    'property_type_encoder': le_property_type,
    'model_name': best_model_name,
    'performance_metrics': results[best_model_name]
}

# Save artifacts
joblib.dump(model_artifacts, '../models/esk_suitability_model.pkl')

print("[SAVED] Model saved successfully!")
print(f"[LOCATION] Location: ../models/esk_suitability_model.pkl")
print(f"[MODEL] Model Type: {best_model_name}")
print(f"[PERFORMANCE] Model Performance: R² = {results[best_model_name]['R²']:.3f}")

# Create prediction function for production
def predict_esk_suitability(property_data):
    """
    Production-ready prediction function
    
    Args:
        property_data (dict): Property features
    
    Returns:
        float: ESK suitability score (0-100)
    """
    # Load model artifacts
    artifacts = joblib.load('../models/esk_suitability_model.pkl')
    model = artifacts['model']
    feature_columns = artifacts['feature_columns']
    
    # Prepare features (simplified for demo)
    features = [property_data[col] for col in feature_columns]
    
    # Make prediction
    prediction = model.predict([features])[0]
    
    return max(0, min(100, prediction))  # Ensure 0-100 range

print("\n[FUNCTION] Production prediction function created!")
print("[READY] Ready for Streamlit app integration")

[SAVED] Model saved successfully!
[LOCATION] Location: ../models/esk_suitability_model.pkl
[MODEL] Model Type: LightGBM
[PERFORMANCE] Model Performance: R² = 0.976

[FUNCTION] Production prediction function created!
[READY] Ready for Streamlit app integration


---
## 📋 Notebook Summary

### ✅ Completed Tasks:
1. **Data Collection**: Generated 1000 housing records for analysis
2. **EDA**: Comprehensive exploratory data analysis with visualizations
3. **Feature Engineering**: Created derived features and encoded categoricals
4. **Model Training**: Trained and compared Random Forest, XGBoost, and LightGBM
5. **Model Evaluation**: Performance metrics and validation
6. **Feature Analysis**: Importance ranking and business insights
7. **Production Export**: Saved model for deployment

### 🎯 Key Findings:
- **Best Model**: {best_model_name} with R² score of {results[best_model_name]['R²']:.3f}
- **Key Factors**: Distance to ESK, property features, and neighborhood matter most
- **Business Value**: Model can predict ESK suitability with high accuracy

### 🚀 Next Steps:
- Integrate model into Streamlit dashboard
- Implement real-time predictions
- Deploy to Streamlit Cloud
- Gather user feedback for model improvement

---
*This notebook demonstrates the complete ML pipeline for the ESKAR Housing Finder project, fulfilling Code Institute PP5 requirements for data analysis, modeling, and business insights.*